<a href="https://colab.research.google.com/github/rajanskumarsoni/Machine-Learning/blob/master/MovieRatingPredictionFactorizationMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
import numpy as np
import pickle
import copy
import warnings
from tensorflow.python.ops import rnn_cell_impl
import tensorflow as tf #machine learningt
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import drive
drive.mount('/content/drive')
np.random.seed(1234)
pd.options.display.max_columns = 10 
pd.options.display.width = 20
pd.options.display.max_rows = 10
train = pd.read_csv('/content/drive/My Drive/prml/train.csv')
test = pd.read_csv('/content/drive/My Drive/prml/test.csv')
validation = pd.read_csv('/content/drive/My Drive/prml/validation.csv')
movies = pd.read_csv('/content/drive/My Drive/prml/movies.csv')
sub = pd.read_csv('/content/drive/My Drive/prml/dummy_submission.csv')
genome_scores = pd.read_csv('/content/drive/My Drive/prml/genome_scores.csv.zip')
genome_attributes = pd.read_csv('/content/drive/My Drive/prml/genome_attributes.csv')
# print(genome_scores)
# print(genome_attributes)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

results = set()
movies['genres'].str.split('|').apply(results.update)

keys = results
values = [0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19]
dictionarys = dict(zip(keys, values))


genreDictionary = {}
count = 0
mylist = []
for movie in movies['genres']:
  
  m= movie.split('|')
  x = [0] * 20
  for k in m:
#     print(k)
    p = dictionarys[k]
#     print(p)
    x[p] = 1
  genreDictionary[count] = x
  count = count +1
  mylist.append(x)


from sklearn.metrics.pairwise import pairwise_kernels
n= pairwise_kernels(mylist, metric='cosine')
x = np.array(n)
p = np.argsort(x)
z = np.flip(p,axis= 1)
mbest = z[:,0:100]
    
   
    
    
    
    


In [0]:

atrix = genome_scores.pivot('movieId','attributeId','relevance')
# print(atrix)
from sklearn.metrics.pairwise import pairwise_kernels
n= pairwise_kernels(atrix.values, metric='cosine')
x = np.array(n)
p = np.argsort(x)
z = np.flip(p,axis= 1)
best_similar_movies = 50
kbest = z[:,0:best_similar_movies]
# print(kbest[1])
indexTomovie ={}
movieToindex = {}
cheeku = sorted(set(genome_scores['movieId']))
# print(cheeku)
letscount = 0
for cheek in cheeku:
  indexTomovie[letscount] = cheek
  movieToindex[cheek] = letscount
  letscount= letscount + 1
  
  


In [4]:
##########################################################Latent code

valdi = validation
valdi.drop(['timestamp'], axis=1)
train = train.append(valdi)
trainx = train['userId']
trainy = train['movieId']
uniquex= trainx.unique()
uniquey= trainy.unique()
uid = train['userId']
mid = train['movieId']
rating = train['rating']
uidvalid = validation['userId']
midvalid = validation['movieId']
ratingvalid = validation['rating']
uitest = test['userId']
mitest = test['movieId']
# matrix = pd.concat([train,test]).pivot('userId','movieId','rating')

# movie_means = matrix.mean()
userset = set(trainx)
movieset = set(trainy)





/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [5]:
k = 70
p= 0.01*np.random.rand(np.max(uniquex)+1,k)
q = 0.01*np.random.rand(np.max(uniquey)+1,k)
bu= 0.01*np.random.rand(np.max(uniquex)+1,1)
bm = 0.01*np.random.rand(np.max(uniquey)+1,1)

epochs = 12
alpha = 0.005
beta = .02
mu = np.mean(rating.values)

for epoch in range(epochs):
  mse=0
  unknown=0
  for u,m,r in zip(uid,mid,rating):
   
    e = r - (np.dot(p[u],q[m])+mu + bu[u] + bm[m])
    p[u] = p[u] + alpha*(2*e*q[m]-beta*p[u])
    q[m] = q[m]  + alpha*(2*e*p[u]-beta*q[m])
    bu[u] = bu[u] +alpha*(e-beta*bu[u])
    bm[m] = bm[m] +alpha*(e-beta*bm[m])
#     mu = mu + alpha*(e - beta(bu[u]+bm[m]))
    mse += (r - (np.dot(p[u],q[m])+mu + bu[u] + bm[m]))**2
  
  print("epoch"+str(epoch)+" MSE "+str(mse/float(len(train))))
  validmse = 0
  cnt=0
  for u,m,r in zip(uidvalid,midvalid,ratingvalid):

    if u not in userset and m not in movieset:
      if m in cheeku:
        summ = 0
        m = movieToindex[m]
        readylist = [indexTomovie[x] for x in kbest[m]]
        bstcount = 0
        for bstmov in readylist:
          if bstmov in movieset:
            summ = summ + bm[bstmov]
            bstcount = bstcount + 1
        
        mymean = summ/bstcount
        validmse+= (r-mymean-mu)**2
      else:
        validmse+= (r-mu)**2
      
    elif u not in userset:
      validmse += (r - mu  - bm[m])**2
    elif m not in movieset:
      if m in cheeku:
        summ = 0
        m = movieToindex[m]
        readylist = [indexTomovie[x] for x in kbest[m]]
        bstcount = 0
        for bstmov in readylist:
          if bstmov in movieset:
            summ = summ + bm[bstmov]
            bstcount = bstcount + 1
        
        mymean = summ/bstcount
        validmse += (r - mu  - bu[u]-mymean)**2
      else:
        validmse += (r - mu  - bu[u])**2
    else:
      validmse += (r - (np.dot(p[u],q[m])+mu + bu[u] + bm[m]))**2

  
   
  print("epoch"+str(epoch)+" MSE "+str(mse/float(len(train)))+"valid MSE "+ str(validmse/float(len(validation)))+str(unknown)+"unknown present")





epoch0 MSE [0.72714139]
epoch0 MSE [0.72714139]valid MSE [0.63029425]0unknown present
epoch1 MSE [0.65262243]
epoch1 MSE [0.65262243]valid MSE [0.58976864]0unknown present
epoch2 MSE [0.6201583]
epoch2 MSE [0.6201583]valid MSE [0.5558933]0unknown present
epoch3 MSE [0.59208844]
epoch3 MSE [0.59208844]valid MSE [0.52281339]0unknown present
epoch4 MSE [0.56420418]
epoch4 MSE [0.56420418]valid MSE [0.49339024]0unknown present
epoch5 MSE [0.53778302]
epoch5 MSE [0.53778302]valid MSE [0.46842303]0unknown present
epoch6 MSE [0.51455696]
epoch6 MSE [0.51455696]valid MSE [0.4471876]0unknown present
epoch7 MSE [0.49402839]
epoch7 MSE [0.49402839]valid MSE [0.42774488]0unknown present
epoch8 MSE [0.47424555]
epoch8 MSE [0.47424555]valid MSE [0.4090582]0unknown present
epoch9 MSE [0.45455197]
epoch9 MSE [0.45455197]valid MSE [0.39097721]0unknown present
epoch10 MSE [0.43509467]
epoch10 MSE [0.43509467]valid MSE [0.37349074]0unknown present
epoch11 MSE [0.41621868]
epoch11 MSE [0.41621868]valid MS

In [0]:
fianlPrediction = []
count =int(0)
for u,m in zip(uitest,mitest):
  rett = []
  if u not in userset and m not in movieset:
    if m in cheeku:
        summ = 0
        m = movieToindex[m]
        readylist = [indexTomovie[x] for x in kbest[m]]
        bstcount = 0
        for bstmov in readylist:
          if bstmov in movieset:
            summ = summ + bm[bstmov]
            bstcount = bstcount + 1
        
        mymean = summ/bstcount
  
        ratee = mymean+mu
        rett.append(int(count))
        rett.append(ratee[0])
        fianlPrediction.append(rett)
        count = count +1
        
    else:
      ratee = mu
      rett.append(int(count))
      rett.append(ratee)
      fianlPrediction.append(rett)
      count = count +1
  elif u not in userset:
    ratee =  mu+ bm[m]
    rett.append(int(count))
    rett.append(ratee[0])
    fianlPrediction.append(rett)
    count = count +1
  elif m not in movieset:
    if m in cheeku:
        summ = 0
        m = movieToindex[m]
        readylist = [indexTomovie[x] for x in kbest[m]]
        bstcount = 0
        for bstmov in readylist:
          if bstmov in movieset:
            summ = summ + bm[bstmov]
            bstcount = bstcount + 1
        
        mymean = summ/bstcount
        
        ratee = mu  +bu[u]+mymean
        rett.append(int(count))
        rett.append(ratee[0])
        fianlPrediction.append(rett)
        count = count +1
    else:

      ratee = mu  +bu[u]
      rett.append(int(count))
      rett.append(ratee[0])
      fianlPrediction.append(rett)
      count = count +1
  else:
    ratee = (np.dot(p[u],q[m])+mu + bu[u] + bm[m])
    rett.append(int(count))
    rett.append(ratee[0])
    fianlPrediction.append(rett)
    count = count +1
    
   
f2 = pd.DataFrame(fianlPrediction, columns=['Id','Prediction'])
f2.to_csv('/content/drive/My Drive/prml/latent.csv', index = False)



In [7]:
!pip install pydrive